<img width="8%" alt="OWID.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/OWID.png" style="border-radius: 15%">

# OWID - Visualize greenhouse gas per capita
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OWID+-+Visualize+greenhouse+gas+per+capita:+Error+short+description">Bug report</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Last update:** 2023-07-31 (Created: 2023-07-31)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's greenhouse gas emissions per capita data.

**References:**
- https://github.com/owid/owid-datasets/tree/master/datasets/GHG%20Emissions%20by%20Country%20and%20Sector%20(CAIT%2C%202021)
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [ ]:
try:
    import dash
    import os
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup Variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel
- `title`: App title

In [ ]:
DASH_PORT = 8050
url = "https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/CO2%20emissions%20by%20sector%20(CAIT%2C%202021)/CO2%20emissions%20by%20sector%20(CAIT%2C%202021).csv"
title = "Greenhouse gas emission per capita"

## Model

### Initialize Dash app
The `os.environ.get("JUPYTERHUB_USER")` is used to access the environment variable `JUPYTERHUB_USER` already stored into your Naas Lab.

In [ ]:
app = dash.Dash(
    title=title,
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

#app = dash.Dash(title = "Greenhouse gas emission per capita") if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_csv(url, header = 0) 
entity = contents_df["Entity"]
year = contents_df["Year"]
energy = contents_df["Energy (per capita)"]# sort the data
industry = contents_df["Industry (per capita)"]
total = contents_df["Total excluding LUCF (per capita)"]

### Create a new dataframe

In [ ]:
categories = []
categories1 = []
categories2 = []
for i in range(len(entity)):
    categories.append("Energy")
    categories1.append("Industry")
    categories2.append("Total")
new_dict = {"Country": entity,
           "Year": year,
            "Categories": categories,
           "CO2 emission": energy}
new_df = pd.DataFrame(data=new_dict)
new_dict1 = {"Country": entity,
           "Year": year,
            "Categories": categories1,
           "CO2 emission": industry}
new_df1 = pd.DataFrame(data=new_dict1)
new_dict2 = {"Country": entity,
           "Year": year,
            "Categories": categories2,
           "CO2 emission": total}
new_df2 = pd.DataFrame(data=new_dict2)
new_df = pd.concat([new_df, new_df1, new_df2])

### Create Dash App

In [ ]:
app.layout = html.Div(
    [
        html.H4("CO2 emission per capita from 1990 to 2018 in tons"),
        html.P("Select country"),
        dcc.Dropdown(
            id="country",
            options=new_df.Country.unique(),
            value="United States"
        ),
        dcc.RangeSlider(id='slider', min=1990, max=2018, value=[1990, 2018],
               marks={x: str(x) for x in [1990, 1995, 2000, 2005, 2010, 2018]}),
        dcc.Graph(id="CO2 emission", figure={}, style={'display': 'none'})
    ]
)

@callback(
    Output("CO2 emission", 'figure'),
    Output("CO2 emission", 'style'),
    Input('country', 'value'),
    Input('slider', 'value')
)

def update_graph(country, year):
    country_list = [country]
    dff = (new_df["Country"].isin(country_list)) & (new_df["Year"] <= year[1]) & (new_df["Year"] >= year[0]) & (new_df["Year"] <= 2021)
        
    figures = px.line(new_df[dff], x='Year', y="CO2 emission", color = "Categories", markers = True).update_layout(
                                plot_bgcolor='rgba(0, 0, 0, 0)', height= 600)
    styles = {'display': 'block'}   
    return figures, styles


## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")